In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import copy
import numpy as np

from matplotlib import pyplot as plt
from tqdm import tqdm
import networkx as nx
from torch.nn.utils import parameters_to_vector, vector_to_parameters
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [2]:
def generate_connected_graph(cluster_sizes=[100, 100], pin=0.5, pout=0.01, seed=0):
    """Generate a random connected graph"""
    probabilities = np.array([[pin, pout], [pout, pin]])
    while True:
        graph = nx.stochastic_block_model(cluster_sizes, probabilities)
        if nx.algorithms.components.is_connected(graph):
            return graph

def visualize_graph(graph):
    nx.draw(graph, with_labels=True, node_size=100, alpha=1, linewidths=10)
    plt.show()

# Parameters
cluster_sizes = [10, 10]
pin = 0.5
pout = 0.2
seed = 0
alpha = 1e-3
lamda = 1e-3
eta = 1e-3
no_users = 30#sum(cluster_sizes)
batch_size = 64
epochs = 1
embedding_dimension = 36358 // 2
iterations = 2000

# Generate a binomial graph with 20 nodes and probability of edge creation p=0.2
G = nx.binomial_graph(n=30, p=0.2, seed=0)
#visualize_graph(graph)

In [3]:
# Metropolis weights 
number_nodes = G.number_of_nodes()
weights = np.zeros([number_nodes, number_nodes])
for edge in G.edges():
  i, j = edge[0], edge[1]
  weights[i - 1][j - 1] = 1 / (1 + np.max([G.degree(i), G.degree(j)]))
  weights[j - 1][i - 1] = weights[i - 1][j - 1]

print(weights)

weights = weights + np.diag(1 - np.sum(weights, axis=0))

metropolis_weights = weights
print(metropolis_weights)


[[0.         0.         0.         0.         0.         0.
  0.         0.16666667 0.         0.         0.         0.
  0.1        0.         0.         0.         0.         0.
  0.14285714 0.         0.         0.         0.         0.
  0.16666667 0.         0.         0.         0.11111111 0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.25       0.14285714 0.
  0.         0.         0.1        0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.14285714 0.
  0.         0.         0.         0.         0.         0.14285714
  0.         0.         0.         0.         0.11111111 0.        ]
 [0.         0.         0.         0.         0.1        0.
  0.1

In [4]:
def degrees(A):
    """Return the degrees of each node of a graph from its adjacency matrix"""
    return np.sum(A, axis=0).reshape(A.shape[0], 1)

def node_degree(n, G):
    cnt = 0
    for i in G.neighbors(n):
        cnt += 1
    return cnt

def get_neighbors(n, G):
    neighbors_list = []
    for i in G.neighbors(n):
        neighbors_list.append(int(i))
    return neighbors_list

In [5]:
# Dataset partitioning
def random_split(X, y, n, seed):
    """Equally split data between n agents"""
    rng = np.random.default_rng(seed)
    perm = rng.permutation(y.size)
    X_split = np.array_split(X[perm], n)  #np.stack to keep as a np array
    y_split = np.array_split(y[perm], n)
    return X_split, y_split



train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')

concatenated_df = pd.concat([train_data, test_data], axis=0)

# Display the concatenated DataFrame
# Replace activities not in the 'keep_activities' list with 'OTHER_ACTIVITIES'

keep_activities = ['SITTING']

concatenated_df['Activity'] = np.where(concatenated_df['Activity'].isin(keep_activities), concatenated_df['Activity'], 'OTHER_ACTIVITIES')

# Split the data into training and testing sets
train_data, test_data = train_test_split(concatenated_df, test_size=0.2, random_state=42)








x_train, y_train = train_data.iloc[:, :-2], train_data.iloc[:, -1:]
x_test, y_test = test_data.iloc[:, :-2], test_data.iloc[:, -1:]
x_train.shape, y_train.shape

x_test, y_test = test_data.iloc[:, :-2], test_data.iloc[:, -1:]
x_test.shape, y_test.shape

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

scaling_data = MinMaxScaler()
x_train = scaling_data.fit_transform(x_train)
X_test = scaling_data.transform(x_test)

train_list = []

for group_value in train_data['subject'].unique():
    print(group_value)
    # Query the DataFrame for the specific group
    group_data = train_data.query(f'subject == {group_value}')
    x_train, y_train = group_data.iloc[:, :-2], group_data.iloc[:, -1:]
    y_train = le.fit_transform(y_train)
    x_train = scaling_data.fit_transform(x_train)

    
    # Extract the 'value' column and convert it to a NumPy array
    train_list.append((x_train, y_train))




no_features = X_test.shape[1]

#X, y = random_split(X_train, y_train, no_users, 1234)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


5
19
13


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


4
6
22


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to

...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


21
30
9


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to

28
8
12
11


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
y_test

array([0, 0, 0, ..., 1, 0, 0])

In [7]:
datapoints = {}
count = 0


scaler = [1.0, -1.0]

noise_sd = 0.001
for i in range(no_users):
    #features = np.random.normal(loc=0.0, scale=1.0, size=(m, n))
    #label = np.dot(features, W[i ]) + np.random.normal(0,noise_sd)
    data = train_list[i][0]
    #data[:, 0:no_features//2] *= scaler[i]
    datapoints[count] = {
            'features': data,
            'degree': node_degree(i, G),
            'label': train_list[i][1].reshape(-1, 1),
            'neighbors': get_neighbors(i, G),
            #'exact_weights': torch.from_numpy(W[i])
        }
    count += 1

In [8]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = torch.FloatTensor(data)
        self.targets = torch.FloatTensor(targets)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        return x, y
    
    def __len__(self):
        return len(self.data)


In [9]:

# Define the logistic regression model
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_size, output_size, user_id):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, output_size)
        self.user_id = user_id

    def forward(self, x):
        out = self.linear(x)
        return torch.sigmoid(out)

In [10]:
from typing import Iterable, Optional

def grads_to_vector(parameters: Iterable[torch.Tensor]) -> torch.Tensor:
    r"""Convert parameters to one vector

    Args:
        parameters (Iterable[Tensor]): an iterator of Tensors that are the
            parameters of a model.

    Returns:
        The parameters represented by a single vector
    """
    # Flag for the device where the parameter is located
    param_device = None

    vec = []
    for param in parameters:
        # Ensure the parameters are located in the same device
        param_device = param.grad

        vec.append(param_device.view(-1))
    return torch.cat(vec)

In [11]:
input_size = no_features  # Replace with the actual number of features
output_size = 1
model = LogisticRegressionModel(input_size, output_size, 0)

In [12]:



# User-specific information
user_id = 10

# Initialize the MLP model
model = LogisticRegressionModel(input_size, output_size, 0)

# Define a binary cross-entropy loss and an optimizer
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001)

# Create a DataLoader for the dataset
dataloader = DataLoader(MyDataset(datapoints[19]["features"], datapoints[19]["label"]), batch_size=64, shuffle=True)

num_epochs = 2000
for epoch in range(num_epochs):
    for (x, y) in dataloader:
        # Forward pass
        outputs = model(x)

        # Calculate the loss
        loss = criterion(outputs, y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print the loss for every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    

Epoch [10/2000], Loss: 0.5842
Epoch [10/2000], Loss: 0.5990
Epoch [10/2000], Loss: 0.5793
Epoch [10/2000], Loss: 0.5917
Epoch [10/2000], Loss: 0.5864
Epoch [20/2000], Loss: 0.5915
Epoch [20/2000], Loss: 0.5472
Epoch [20/2000], Loss: 0.5585
Epoch [20/2000], Loss: 0.5667
Epoch [20/2000], Loss: 0.5660
Epoch [30/2000], Loss: 0.5734
Epoch [30/2000], Loss: 0.5196
Epoch [30/2000], Loss: 0.5619
Epoch [30/2000], Loss: 0.5479
Epoch [30/2000], Loss: 0.5277
Epoch [40/2000], Loss: 0.5366
Epoch [40/2000], Loss: 0.5124
Epoch [40/2000], Loss: 0.5311
Epoch [40/2000], Loss: 0.5464
Epoch [40/2000], Loss: 0.5262
Epoch [50/2000], Loss: 0.5155
Epoch [50/2000], Loss: 0.5016
Epoch [50/2000], Loss: 0.5334
Epoch [50/2000], Loss: 0.5068
Epoch [50/2000], Loss: 0.5293
Epoch [60/2000], Loss: 0.5475
Epoch [60/2000], Loss: 0.5136
Epoch [60/2000], Loss: 0.4606
Epoch [60/2000], Loss: 0.4845
Epoch [60/2000], Loss: 0.5211
Epoch [70/2000], Loss: 0.4933
Epoch [70/2000], Loss: 0.4776
Epoch [70/2000], Loss: 0.4991
Epoch [70/

Epoch [560/2000], Loss: 0.4168
Epoch [560/2000], Loss: 0.4269
Epoch [560/2000], Loss: 0.3901
Epoch [560/2000], Loss: 0.3509
Epoch [560/2000], Loss: 0.4029
Epoch [570/2000], Loss: 0.3632
Epoch [570/2000], Loss: 0.4251
Epoch [570/2000], Loss: 0.4505
Epoch [570/2000], Loss: 0.3918
Epoch [570/2000], Loss: 0.3415
Epoch [580/2000], Loss: 0.3093
Epoch [580/2000], Loss: 0.4013
Epoch [580/2000], Loss: 0.4260
Epoch [580/2000], Loss: 0.3603
Epoch [580/2000], Loss: 0.5101
Epoch [590/2000], Loss: 0.3302
Epoch [590/2000], Loss: 0.4722
Epoch [590/2000], Loss: 0.4118
Epoch [590/2000], Loss: 0.3764
Epoch [590/2000], Loss: 0.3877
Epoch [600/2000], Loss: 0.3452
Epoch [600/2000], Loss: 0.3574
Epoch [600/2000], Loss: 0.4828
Epoch [600/2000], Loss: 0.3840
Epoch [600/2000], Loss: 0.4121
Epoch [610/2000], Loss: 0.4390
Epoch [610/2000], Loss: 0.4728
Epoch [610/2000], Loss: 0.3516
Epoch [610/2000], Loss: 0.3728
Epoch [610/2000], Loss: 0.3243
Epoch [620/2000], Loss: 0.3674
Epoch [620/2000], Loss: 0.3872
Epoch [6

Epoch [1090/2000], Loss: 0.2569
Epoch [1090/2000], Loss: 0.3953
Epoch [1090/2000], Loss: 0.4653
Epoch [1090/2000], Loss: 0.4302
Epoch [1090/2000], Loss: 0.3451
Epoch [1100/2000], Loss: 0.4018
Epoch [1100/2000], Loss: 0.3810
Epoch [1100/2000], Loss: 0.3917
Epoch [1100/2000], Loss: 0.3841
Epoch [1100/2000], Loss: 0.3295
Epoch [1110/2000], Loss: 0.4237
Epoch [1110/2000], Loss: 0.3224
Epoch [1110/2000], Loss: 0.4640
Epoch [1110/2000], Loss: 0.3606
Epoch [1110/2000], Loss: 0.3126
Epoch [1120/2000], Loss: 0.3555
Epoch [1120/2000], Loss: 0.3462
Epoch [1120/2000], Loss: 0.4059
Epoch [1120/2000], Loss: 0.4265
Epoch [1120/2000], Loss: 0.3579
Epoch [1130/2000], Loss: 0.3932
Epoch [1130/2000], Loss: 0.3944
Epoch [1130/2000], Loss: 0.4126
Epoch [1130/2000], Loss: 0.3293
Epoch [1130/2000], Loss: 0.3621
Epoch [1140/2000], Loss: 0.4299
Epoch [1140/2000], Loss: 0.3173
Epoch [1140/2000], Loss: 0.3630
Epoch [1140/2000], Loss: 0.3923
Epoch [1140/2000], Loss: 0.3951
Epoch [1150/2000], Loss: 0.4231
Epoch [1

Epoch [1620/2000], Loss: 0.3782
Epoch [1620/2000], Loss: 0.4205
Epoch [1630/2000], Loss: 0.4369
Epoch [1630/2000], Loss: 0.3718
Epoch [1630/2000], Loss: 0.4062
Epoch [1630/2000], Loss: 0.3330
Epoch [1630/2000], Loss: 0.2692
Epoch [1640/2000], Loss: 0.3909
Epoch [1640/2000], Loss: 0.4482
Epoch [1640/2000], Loss: 0.3520
Epoch [1640/2000], Loss: 0.3074
Epoch [1640/2000], Loss: 0.3311
Epoch [1650/2000], Loss: 0.3951
Epoch [1650/2000], Loss: 0.3550
Epoch [1650/2000], Loss: 0.3578
Epoch [1650/2000], Loss: 0.3486
Epoch [1650/2000], Loss: 0.3837
Epoch [1660/2000], Loss: 0.4118
Epoch [1660/2000], Loss: 0.3317
Epoch [1660/2000], Loss: 0.3586
Epoch [1660/2000], Loss: 0.3851
Epoch [1660/2000], Loss: 0.3431
Epoch [1670/2000], Loss: 0.3316
Epoch [1670/2000], Loss: 0.4074
Epoch [1670/2000], Loss: 0.4581
Epoch [1670/2000], Loss: 0.2664
Epoch [1670/2000], Loss: 0.3722
Epoch [1680/2000], Loss: 0.3387
Epoch [1680/2000], Loss: 0.4150
Epoch [1680/2000], Loss: 0.4022
Epoch [1680/2000], Loss: 0.2973
Epoch [1

In [13]:
test_loader = DataLoader(MyDataset(X_test, y_test.reshape(-1, 1)), batch_size=64, shuffle=False)

model.eval()
test_loss = 0
correct = 0
total_samples = 0
with torch.no_grad():
    for data, labels in test_loader:
        output = model(data)
        loss = criterion(output, labels)
        test_loss += loss.item() * data.size(0)

        pred = (output >= 0.5).float()
        correct += pred.eq(labels.data.view_as(pred)).sum().item()
        total_samples += labels.size(0)
        print(pred)

test_loss /= len(test_loader.dataset)
test_accuracy = correct / total_samples

print(f'test accuracy {test_accuracy}')

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
     

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
     

In [14]:
'''
# Plotting Accuracies
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)  # 1 row, 2 columns, first subplot
plt.plot(acuracies, label='Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plotting Losses
plt.subplot(1, 2, 2)  # 1 row, 2 columns, second subplot
plt.plot(losses, label='Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()  # Ensures proper spacing between subplots
plt.show()
'''

"\n# Plotting Accuracies\nplt.figure(figsize=(12, 6))\n\nplt.subplot(1, 2, 1)  # 1 row, 2 columns, first subplot\nplt.plot(acuracies, label='Accuracy')\nplt.title('Accuracy Over Epochs')\nplt.xlabel('Epoch')\nplt.ylabel('Accuracy')\nplt.legend()\n\n# Plotting Losses\nplt.subplot(1, 2, 2)  # 1 row, 2 columns, second subplot\nplt.plot(losses, label='Loss')\nplt.title('Loss Over Epochs')\nplt.xlabel('Epoch')\nplt.ylabel('Loss')\nplt.legend()\n\nplt.tight_layout()  # Ensures proper spacing between subplots\nplt.show()\n"

In [15]:
class ClientUpdate(object):
    def __init__(self, dataset, batchSize, alpha, lamda, epochs, projection_list, projected_weights):
        self.train_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=batchSize, shuffle=True)
        #self.learning_rate = learning_rate
        self.epochs = epochs
        self.batchSize = batchSize

    def train(self, model):
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.5)

        e_loss = []
        for epoch in range(1, self.epochs+1):
            train_loss = 0
            model.train()
            for i, (data, labels) in zip(range(1), self.train_loader):
                data, labels = data, labels
                optimizer.zero_grad() 
                output = model(data)  
                loss = criterion(output, labels)
                #loss += mu/2 * torch.norm(client_param.data - server_param.data)**2
                loss.backward()
                grads = grads_to_vector(model.parameters())
                #optimizer.step()
                train_loss += loss.item()*data.size(0)
                weights = parameters_to_vector(model.parameters())
                mat_vec_sum = torch.zeros_like(weights)
                for j in G.neighbors(model.user_id):
                    mat_vec_sum = torch.add(mat_vec_sum, torch.matmul(torch.transpose(projection_list[model.user_id][j], 0, 1), 
                                                         projected_weights[model.user_id][j] - projected_weights[j][model.user_id]))
                
                model_update = parameters_to_vector(model.parameters()) - alpha * (grads + lamda * mat_vec_sum)
                
            vector_to_parameters(parameters=model.parameters(), vec=model_update)
                

            train_loss = train_loss/self.batchSize#len(self.train_loader.dataset) 
            e_loss.append(train_loss)

        total_loss = e_loss#sum(e_loss)/len(e_loss)

        return model.state_dict(), total_loss

In [16]:
# Preparing projection matrices
models = [LogisticRegressionModel(input_size, output_size, i) for i in range(no_users)]
#temp = MLP_Net()
projection_list = []
projected_weights = []

def update_ProjWeight(projection_list, projected_weights, first_run=True):
    #projected_weights = []
    for i in range(no_users):
        neighbors_mat = []
        neighbors_weights = []
        for j in range(no_users):
            if j in G.neighbors(i):
                with torch.no_grad():
                    if first_run == True:
                        row, column = embedding_dimension, parameters_to_vector(models[i].parameters()).size()[0]
                        mat = torch.zeros((row, column))
                        mat.fill_diagonal_(1.0 + 1.0 * float(np.random.randn(1)))
                        neighbors_mat.append(mat)
                        neighbors_weights.append(torch.matmul(mat, parameters_to_vector(models[i].parameters())))
                    else:
                        neighbors_weights.append(torch.matmul(projection_list[i][j], parameters_to_vector(models[i].parameters())))
            else:
                neighbors_mat.append(0)
                neighbors_weights.append(0)
        if first_run == True:
            projection_list.append(neighbors_mat)
        projected_weights.append(neighbors_weights)

update_ProjWeight(projection_list, projected_weights)



RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 40866392 bytes.

In [ ]:
total_params = sum(p.numel() for p in models[0].parameters())
total_weights = sum(p.numel() for p in models[0].parameters() if p.requires_grad)
total_biases = total_params - total_weights

print(f'Total parameters in the model: {total_params}')
print(f'Total weights in the model: {total_weights}')
print(f'Total biases in the model: {total_biases}')

# 52672 no biases

In [ ]:
print(projection_list[0][4].shape)

In [ ]:
def testing(model, dataset, bs, criterion):
    test_loss = 0
    correct = 0
    total_samples = 0

    test_loader = DataLoader(MyDataset(X_test, y_test), batch_size=bs, shuffle=False)
    
    model.eval()

    with torch.no_grad():
        for data, labels in test_loader:
            output = model(data)
            loss = criterion(output, labels)
            test_loss += loss.item() * data.size(0)

            _, pred = torch.max(output, 1)
            correct += pred.eq(labels.data.view_as(pred)).sum().item()
            total_samples += labels.size(0)

    test_loss /= len(test_loader.dataset)
    test_accuracy = correct / total_samples

    return test_loss, test_accuracy

In [ ]:
projection_list[0][6].shape

In [ ]:
#global_model = CNN_Net().cuda()
models = [LogisticRegressionModel(input_size, output_size, i) for i in range(no_users)]
dummy_models = [LogisticRegressionModel(input_size, output_size, i) for i in range(no_users)]

#model.load_state_dict(global_model.state_dict())

criterion = nn.CrossEntropyLoss()

it = 100
train_loss = []
test_loss = []
test_accuracy = []
total_rel_error = []

for curr_round in tqdm(range(1, it+1)):
    w, local_loss = [], []

    
    for i in range(no_users):
        dummy_models[i].load_state_dict(models[i].state_dict())
        local_update = ClientUpdate(dataset=datapoints[i], batchSize=batch_size, alpha=alpha, lamda=lamda, epochs=1, projection_list=projection_list, projected_weights=projected_weights)
        weights, loss = local_update.train(dummy_models[i])
        w.append(weights)
        local_loss.append(loss)
        models[i].load_state_dict(w[i])
        
    
    
    # Update prjection matrix
    
    #print(projection_list[0], projected_weights[0])
    
    for i in range(no_users):
        weights = parameters_to_vector(models[i].parameters())
        for j in G.neighbors(i):
            mat_vec_sum = torch.zeros(embedding_dimension)
            for k in G.neighbors(i):
                mat_vec_sum = torch.add(mat_vec_sum, projected_weights[i][k] - projected_weights[k][i])
            temp_mat = torch.outer(mat_vec_sum, weights).clone()


            projection_list[i][j] = torch.add(projection_list[i][j], -1 * eta * lamda * temp_mat)
                                         
    projected_weights = []                                          
    update_ProjWeight(projection_list, projected_weights, first_run=False)
        
        
        
    
    




          
            

    local_test_acc = []
    local_test_loss = []
    user_rel_error = 0
    for k in range(no_users):
      
        g_loss = testing(models[i], datapoints[i], 50, criterion)
        local_test_loss.append(g_loss)
        #user_rel_error += rel_error(models[i])
    
    
        

    g_loss = sum(local_test_loss) / len(local_test_loss)
    #total_rel_error.append(user_rel_error / no_users)
    
    

    test_loss.append(g_loss)
    #test_accuracy.append(g_accuracy)
    print("Training_loss %2.5f"% (test_loss[-1]))

In [ ]:
#Training_loss 5.33078 with no communication

In [ ]:
#plot.plot(test_loss)
parameters_to_vector(models[19].parameters())

In [ ]:
for j in G.neighbors(0):
    print(j)

In [ ]:
parameters_to_vector(models[0].parameters())

In [ ]:
projection_list[0]

In [ ]:
projected_weights[0]

In [ ]:
test_loss = np.array(test_loss)
total_rel_error = np.array(total_rel_error)

In [ ]:
print(test_loss)

In [ ]:
np.save( 'training_loss_sheave_fml_lambda' + str(lamda).replace('.', '_') + '_pout' + str(pout).replace('.', '_'), test_loss)
#np.save('relative_error_sheave_fml' + str(lamda).replace('.', '_'), total_rel_error)

In [ ]:
'training_loss_sheave_fml' + str(lamda).replace('.', '_'), test_loss